In [1]:
%reload_ext autoreload
%autoreload 2
from plot_logger import *

In [5]:
query = f"""
        select
        to_timestamp(timestamp) as fecha,
        device,
        metric,
        value
        from 'csv/*/*.csv'
        """

duckdb.sql(query).df()

,fecha,device,metric,value
0,2024-12-10 21:23:38+01:00,solar,power,0.00
1,2024-12-10 21:23:38+01:00,humedad suelo,temperature,16.50
2,2024-12-10 21:23:38+01:00,humedad suelo,humidity,35.02
3,2024-12-10 21:23:38+01:00,humedad suelo,ec,275.00
4,2024-12-10 21:23:38+01:00,pH,pH,8.40
...,...,...,...,...
2076,2024-12-13 23:37:25+01:00,temp_interior,temperature,5.50
2077,2024-12-13 23:37:25+01:00,temp_exterior,humidity,51.80
2078,2024-12-13 23:37:25+01:00,temp_exterior,temperature,3.40
2079,2024-12-13 23:37:25+01:00,co2,co2,670.00


In [3]:
hours = 24
plot('vpd', devices=['temp_interior', 'temp_exterior'], height=600, hours=hours)[0].show()
plot('temperature', hours=hours)[0].show()
plot('humidity', hours=hours)[0].show()
plot('presion', hours=hours)[0].show()
plot('co2')[0].show()
multiplot(metrics = ['temperature', 'vpd', 'humidity'], devices=['temp_interior'], height=800)[0].show()
multiplot(metrics = ['temperature', 'vpd', 'humidity'], devices=['temp_exterior'], height=800)[0].show()
plot('power', hours=hours)[0].show()

Temperature:
- To raise VPD: increase temperature
- To lower VPD: decrease temperature

Humidity
- To raise VPD: decrease humidity (turn dehumidifier on)
- To lower VPD: increase humidity (turn dehumidifier off)

In [78]:
query = f"""
                select
                to_timestamp(timestamp) as fecha,
                device,
                metric,
                value
                from 'csv/*/*.csv'
                where to_timestamp(timestamp) >= current_timestamp - interval 24 hour
                and metric = 'action'
                and device = 'ventilador 1'
                order by timestamp desc
                """
        
duckdb.sql(query).df()

,fecha,device,metric,value
0,2024-12-07 19:26:49+01:00,ventilador 1,action,1.0
1,2024-12-07 19:16:33+01:00,ventilador 1,action,0.0
2,2024-12-07 19:10:22+01:00,ventilador 1,action,1.0
3,2024-12-07 18:56:57+01:00,ventilador 1,action,0.0
4,2024-12-07 18:47:41+01:00,ventilador 1,action,1.0
5,2024-12-07 18:36:26+01:00,ventilador 1,action,0.0
6,2024-12-07 18:32:21+01:00,ventilador 1,action,1.0
7,2024-12-07 15:00:11+01:00,ventilador 1,action,0.0
8,2024-12-07 12:09:24+01:00,ventilador 1,action,1.0
9,2024-12-07 12:08:22+01:00,ventilador 1,action,0.0


In [58]:
from dash import Dash, html, dcc, callback, Input, Output
from plot_logger import *

app = Dash()

app.layout = html.Div([
    html.Div(
        [
            dcc.RadioItems(
                options=['VPD', 'Temperature', 'Humidity'], 
                value='VPD',
                id='metric_selector', 
                inline=True
            ),
            dcc.Checklist(
                options=['temp_interior', 'temp_exterior', 'suelo', 'humedad suelo'],
                value=['temp_interior', 'temp_exterior'],
                inline=True,
                id='device_selector'
            )
        ],
        style={
            'display': 'flex',
            'alignItems': 'center',
            'gap': '50px',
            'margin-left': '30px',
            'margin-top': '10px'
        }
    ),
    html.Div(
        dcc.Slider(
            id="hours_selector",
            min=1,
            max=25,
            value=25,
            step=1
        ),
        style={
            'alignItems': 'left',
            'maxWidth': '600px',
            'margin-left': '30px',
            'margin-top': '10px'
        }
    ),
    dcc.Graph(
        id = 'chart'
        )
    ])
    

@callback(
    Output(component_id='chart', component_property='figure'),
     [
    Input(component_id='metric_selector', component_property='value'),
    Input(component_id='device_selector', component_property='value'),
    Input(component_id='hours_selector', component_property='value'),
     ]
)
def update_chart(input_value, devices, hours):
    return plot(input_value.lower(), devices=devices, height=600, hours=hours)[0]

app.run(
    debug=False,
    host='0.0.0.0',
    port=8050
    )